In [1]:
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


def conv_dw(inp, oup, stride):
    # 深度可分离卷积层
    return nn.Sequential(
        # depthwise层：卷积计算层，计算输入inp卷积
        nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
        nn.BatchNorm2d(inp),
        nn.ReLU6(inplace=True),
        # pointwise层：卷积合并层，将计算的inp卷积与oup合并
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
    )


def conv_bn(inp, oup, stride):
    # 标准卷积层
    return nn.Sequential(
        # 二维卷积，kernal_size(卷积核)=3*3, padding(边界填充数)=1
        # stride(卷积窗口滑动步长)
        # bias=False(卷积核只有权重系数，没有偏执系数，可减少训练参数)
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        # 归一化层，采取batchnorm归一化方法（不改变输入输出大小，只进行归一化操作）
        nn.BatchNorm2d(oup),
        # 激活函数
        nn.ReLU6(inplace=True)
    )


class ConvNet(nn.Module):
    # 初始化手写识别网络模型
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        # 标准3x3卷积层，名字为conv1，input=3，output=8，stride=1(输出特征宽高不变)
        # 3x64x64 --> 8x64x64 （输入维度 --> 输出维度）
        self.conv1 = conv_bn(3, 8, 1)
        # 标准3x3卷积层，名字为conv2，input=8，output=16，stride=1
        # 8x64x64 --> 16x64x64
        self.conv2 = conv_bn(8, 16, 1)
        # 深度可分离卷积层，名字为conv3，input=16，output=32，stride=1
        # 16x64x64 --> 32x64x64
        self.conv3 = conv_dw(16, 32, 1)
        # 深度可分离卷积层，名字为conv4，input=32，output=32，stride=2(此时输出特征宽高减半)
        # 32x64x64 --> 32x32x32
        self.conv4 = conv_dw(32, 32, 2)
        # 深度可分离卷积层，名字为conv5，input=32，output=64，stride=1
        # 32x32x32 --> 64x32x32
        self.conv5 = conv_dw(32, 64, 1)
        # 深度可分离卷积层，名字为conv6，input=64，output=64，stride=2(此时输出特征宽高减半)
        # 64x32x32 --> 64x16x16  下面依此类推
        self.conv6 = conv_dw(64, 64, 2)  # 64x16x16
        self.conv7 = conv_dw(64, 128, 1)  # 128x16x16
        self.conv8 = conv_dw(128, 128, 1)  # 128x16x16
        self.conv9 = conv_dw(128, 128, 1)  # 128x16x16
        self.conv10 = conv_dw(128, 128, 1)  # 128x16x16
        self.conv11 = conv_dw(128, 128, 1)  # 128x16x16
        self.conv12 = conv_dw(128, 256, 2)  # 256x8x8
        self.conv13 = conv_dw(256, 256, 1)  # 256x8x8
        self.conv14 = conv_dw(256, 256, 1)  # 256x8x8
        self.conv15 = conv_dw(256, 512, 2)  # 512x4x4
        self.conv16 = conv_dw(512, 512, 1)  # 512x4x4
        # 模型的分类器，分类器由如下网络层组合，输出为分类的类别
        self.classifier = nn.Sequential(
            # Linear为全连接层，将512*4*4的数据传入，输出1024
            nn.Linear(512*4*4, 1024),
            # Dropout层，功能：每个神经元都有0.2的概率被失效（即该神经元不起作用）
            # Dropout层，作用：为了防止过拟合
            nn.Dropout(0.2),
            # ReLU 激活函数，inplace=True表示进行覆盖运算，即后面的计算值覆盖前面的
            nn.ReLU(inplace=True),
            # 最后通过一个全连接层，输入1024数据，输出汉字字符个数（num_classes）
            nn.Linear(1024, num_classes),
        )
        # 神经网络权重初始化（不同层不同初始化方法，看下面的注释）
        self.weight_init()

    # 网络模型的前向训练
    def forward(self, x):
        x1 = self.conv1(x)  # x为输入数据（即图像数据），x1为经过卷积层conv1处理后的特征数据
        x2 = self.conv2(x1)  # x1为上一层输出数据，传递到下层作为输入数据，经过卷积层conv2处理后输出数据为x2，以此类推
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        x7 = self.conv7(x6)
        x8 = self.conv8(x7)
        x9 = self.conv9(x8)
        # 将conv8卷积后的特征和conv9卷积后的特征累加作为特征，经激活函数relu处理后赋值给x9
        # 目的：提高特征梯度，即使图像特征更明显
        x9 = F.relu(x8 + x9)
        x10 = self.conv10(x9)
        x11 = self.conv11(x10)
        x11 = F.relu(x10 + x11)
        x12 = self.conv12(x11)
        x13 = self.conv13(x12)
        x14 = self.conv14(x13)
        x14 = F.relu(x13 + x14)
        x15 = self.conv15(x14)
        x16 = self.conv16(x15)
        # 修改x16数据tensor形状为x16.size(0)*?, 此时？由x16的数据量自己推算
        # 如x16的size为3*3*2，则x = x16.view(x16.size(0), -1)后，x的size为3*6
        x = x16.view(x16.size(0), -1)
        # 经分类器输出类别tensor数据x
        x = self.classifier(x)
        return x

    def weight_init(self):
        # 权重初始化，根据不同的层类别进行不同的初始化方法
        for layer in self.modules():
            self._layer_init(layer)

    def _layer_init(self, m):
        # 使用isinstance来判断m属于什么类型的网络层
        if isinstance(m, nn.Conv2d):
            # 二维卷积层，初始化方法为：均值为0，方差为np.sqrt(2. / n)的正态分布进行随机初始化
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, np.sqrt(2. / n))
        elif isinstance(m, nn.BatchNorm2d):
            # batchnorm层的weight和bias为可学习变量，将weight初始化为1，bias为0
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            # 全连接层，weight初始化：均值为0，方差为0.01的正态分布进行随机初始化，bias初始化为0
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()


if __name__ == "__main__":
    model = ConvNet(3755).cuda()
    summary(model, input_size=(3, 64, 64), device='cuda')

ImportError: No module named 'torchsummary'